In [6]:
import mlflow
import requests
import json
import time
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, AzureCliCredential, InteractiveBrowserCredential
from datetime import datetime
from pathlib import Path

from mlflow.tracking import MlflowClient
from mlflow.utils.rest_utils import http_request
import uuid

subscription_id = '96aede12-2f73-41cb-b983-6d11a904839b'
resource_group = 'promptflow'
workspace_name = 'promptflow-eastus'

# subscription_id = '96aede12-2f73-41cb-b983-6d11a904839b'
# resource_group = 'yonwa-rg'
# workspace_name = 'yonwa-trace-ai-studio'

ml_client = MLClient(credential=AzureCliCredential(),
                        subscription_id=subscription_id,
                        resource_group_name=resource_group,
                        workspace_name=workspace_name)

base_endpoint = ml_client.workspaces.get(ml_client.workspace_name).discovery_url.replace("discovery", "")
url = (
    f"history/v1.0"
    f"/subscriptions/{ml_client.subscription_id}"
    f"/resourceGroups/{ml_client.resource_group_name}"
    f"/providers/Microsoft.MachineLearningServices"
    f"/workspaces/{ml_client.workspace_name}"
)
endpoint_url = base_endpoint + url
print(endpoint_url)

https://eastus.api.azureml.ms/history/v1.0/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-eastus


In [10]:
credential=AzureCliCredential()
# credential=InteractiveBrowserCredential()
token = credential.get_token("https://management.azure.com/.default").token

headers = {  
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json',
} 

### Get run from run history

In [36]:

run_id = "web_classification_variant_0_20240407_191438_307497"    # eastus
# run_id = "web_classification_default_20230809_163800_975156"    # hod-pflow
url = f"{endpoint_url}/runs/{run_id}"

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print(f"Successfully get run from run history")
    with open("./download/rh_batch_run_ai_project.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

Successfully get run from run history


### Modify run in run history

In [ ]:
class MyClass:
    a = 5

# /experimentids/42caf8b8-562b-47f6-b838-0ae2aabb1dc1
run_id = "web_classification_default_20230815_103013_448837"    # eastus
url = f"{endpoint_url}/runs/{run_id}/modify"

payload = {
    # "runId": run_id,
    "hidden": False,
    "tags": {
        "test_tag": {"a": 123}
    },
    "description": "",
    "displayName": "hod-run",
}
response = requests.patch(url, headers=headers, json=payload)

if response.status_code == 200:
    print(f"Successfully updated run.")
    with open("./update/rh_batch_run_resp.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

### Get run data from run history

In [26]:

run_id = "web_classification_variant_0_20240407_190159_783736"    # eastus
# run_id = "web_classification_default_20230809_163800_975156"    # hod-pflow
url = f"{endpoint_url}/rundata"

payload = {
    "runId": run_id,
    "selectRunMetadata": True,
    "selectRunDefinition": True,
    "selectJobSpecification": True,
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    print(f"Successfully get run from run history")
    with open("./download/rh_batch_run_data_local2cloud.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

Code: 404, Reason: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "Run runId=web_classification_variant_0_20240407_190159_783736 was not found",
    "messageFormat": "Run {runId} was not found",
    "messageParameters": {
      "runId": "runId=web_classification_variant_0_20240407_190159_783736"
    },
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null,
    "additionalInfo": null
  },
  "correlation": {
    "operation": "f682c1a563200cc6241469dff073da61",
    "request": "7a1d82af455620bf"
  },
  "environment": "eastus",
  "location": "eastus",
  "time": "2024-04-07T11:04:00.3547779+00:00",
  "componentName": "run-history",
  "statusCode": 404
}


### Get run from PFS

In [3]:
run_id = "6ce7ef3a-44ed-47c6-abe8-f4e760f67a78"
pfs_endpoint_url = endpoint_url.replace("history/v1.0", "flow/api")
url = f"{pfs_endpoint_url}/BulkRuns/{run_id}"

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print(f"Successfully get runs from PFS")
    with open("./pfs_run_info.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

Successfully get child runs from PFS


### Get run from index service

In [ ]:
run_id = "c619f648-c809-4545-9f94-f67b0a680706"
index_endpoint_url = endpoint_url.replace("/history", "/index")
url = f"{index_endpoint_url}/entities"

payload = {
    "filters": [{
        "field": "type",
        "operator": "eq",
        "values": ["runs"]
    }, {
        "field": "annotations/archived",
        "operator": "eq",
        "values": ["false"]
    },
        {
            "field": "properties/runId",
            "operator": "eq",
            "values": [run_id]
        }
    ],
    "order": [{
        "direction": "Desc",
        "field": "properties/startTime"
    }
    ],
    "pageSize": 50,
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    print(f"Successfully get run info from index service")
    with open("./download/index_batch_run.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

### List runs from index service

In [ ]:
index_endpoint_url = endpoint_url.replace("/history", "/index")
url = f"{index_endpoint_url}/entities"

payload = {
    "filters": [
        {
            "field": "type",
            "operator": "eq",
            "values": ["runs"]
        },
        {
            "field": "annotations/archived",
            "operator": "eq",
            "values": ["false"]
        },
        {
            "field": "properties/runType",
            "operator": "contains",
            "values": [
                "azureml.promptflow.FlowRun",
                "azureml.promptflow.EvaluationRun",
            ]
        }
    ],
    "freeTextSearch": "",
    "order": [
        {
            "direction": "Desc",
            "field": "properties/creationContext/createdTime"
        }
    ],
    # index service can return 100 results at most
    "pageSize": 50,
    "skip": 0,
    "includeTotalResultCount": True,
    "searchBuilder": "AppendPrefix"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    print(f"Successfully listed runs from index service")
    with open("./list_entities_50.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

### Get metrics from metric service

In [ ]:
run_id = "groundedness_eval_default_20230821_191944_818788"
index_endpoint_url = endpoint_url.replace("/history/v1.0", "/metric/v2.0")
url = f"{index_endpoint_url}/runs/{run_id}/lastvalues"

response = requests.post(url, json={}, headers=headers)

if response.status_code == 200:
    print(f"Successfully got metrics from metric service")
    with open("./metrics_cle.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

### List flows from index service

In [ ]:
index_endpoint_url = endpoint_url.replace("/history", "/index")
url = f"{index_endpoint_url}/entities"

payload = {
    "filters": [
        {
            "field": "type",
            "operator": "eq",
            "values": ["flows"]
        },
        # {
        #     "field": "annotations/archived",
        #     "operator": "eq",
        #     "values": ["true"]
        # },
        {
            "field": "properties/creationContext/createdBy/userTenantId",
            "operator": "eq",
            "values": ["72f988bf-86f1-41af-91ab-2d7cd011db47"]
        },
        {
            "field": "properties/creationContext/createdBy/userObjectId",
            "operator": "eq",
            "values": ["c05e0746-e125-4cb3-9213-a8b535eacd79"]
        }
    ],
    "freeTextSearch": "",
    "order": [
        {
            "direction": "Desc",
            "field": "properties/creationContext/createdTime"
        }
    ],
    # index service can return 100 results at most
    "pageSize": 10,
    "skip": 0,
    "includeTotalResultCount": True,
    "searchBuilder": "AppendPrefix"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    print(f"Successfully listed runs from index service")
    with open("./flow_list_archived.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

### Get snapshot sas token from content service

In [ ]:
snapshot_id = "194d5c58-3176-4311-88f0-e01a31a5f706"
index_endpoint_url = endpoint_url.replace("/history/v1.0", "/content/v2.0")
url = f"{index_endpoint_url}/snapshots/sas"

payload = {
    "snapshotOrAssetId": snapshot_id,
    # "path": "string"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    print(f"Successfully got metrics from metric service")
    with open("./download/snapshot_sas_resp.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

### Get merics from metric service

In [17]:
run_id = "805dffc1-df00-4722-87f1-3e774188458e"    # eastus
run_id = "0db9f8df-5f13-4798-93f9-d5157acb0dcb"
index_endpoint_url = endpoint_url.replace("/history/v1.0", "/metric/v2.0")
url = f"{index_endpoint_url}/runs/{run_id}/lastvalues"

response = requests.post(url, headers=headers, json={})

if response.status_code == 200:
    print(f"Successfully get run metrics from metric service")
    with open("./download/run_metrics.json", "w") as f:
        json.dump(response.json(), f, indent=4)
else:
    print(f"Code: {response.status_code}, Reason: {response.text}")

Successfully get run metrics from metric service


In [ ]:
import re

OUTPUT_PORTAL_PATTERN = re.compile(r"azureml://.*?/data/(?P<name>.*?)/versions/(?P<version>.*?)$")
target = 'azureml://locations/eastus/workspaces/3e123da1-f9a5-4c91-9234-8d9ffbb39ff5/data/azureml_web_classification_default_20230809_120434_491077_output_data_flow_outputs/versions/1'

match = OUTPUT_PORTAL_PATTERN.match(target)
if match:
    print(match.group('name'))
    print(match.group('version'))
print(match.groups())

In [ ]:
from pathlib import Path
a = Path(".").resolve().absolute()
print(str(a))
print(a.as_posix())

## Test promptflow schema

In [ ]:
import json
from marshmallow_jsonschema import JSONSchema  
from promptflow._sdk.schemas._flow_dag import FlowDagSchema

json_schema = JSONSchema().dump(FlowDagSchema(context={"base_path": "."}))
with open("./flow_dag_schema.json", "w") as f:
    json.dump(json_schema, f, indent=4)

# Temprary test

In [11]:
a = Path("./chat_group123.ipynb")
print(a.is_file())

False


In [36]:
yaml_file = "./flows/web_classification/flow.dag.yaml"

with open(yaml_file, encoding="utf-8") as f:
    yaml1 = yaml.safe_load(f)

with open(yaml_file, encoding="utf-8") as f:
    yaml2 = ryaml.load(f)

In [2]:
from pathlib import Path
a = r'C:\Users\hod\.promptflow\.runs\91745026-c298-46f2-8949-0fa6810813ee\flow_outputs\output.jsonl'
a = Path(a)
print(a.is_file())
print(a.stat().st_size)

True
0
